In [1]:
import os
import pandas as pd
import janitor
from IPython.display import display
from inequality.gini import Gini
from scipy.stats import circmean
import warnings

warnings.filterwarnings("ignore")

import numpy as np

FP_IND_DOMAIN = "../data/yg_ind_domain.csv.gz"
FP_VISITS_DATA = "../data/6.3m-ind-domain-data.csv.gz"
FP_VT_LABELS = "../data/yg_virustotal_dat.csv"
FP_IND_DEMO = "../data/profile.csv"
FILEPATHS = [value for name, value in globals().items() if name.startswith("FP_")]
for file_path in FILEPATHS:
    try:
        assert os.path.exists(file_path), f"File not found: {file_path}"
    except AssertionError:
        print(f"File not found: {file_path}.")

In [2]:
df_visits = (
    pd.read_csv(FP_VISITS_DATA)
    # ==================================================
    # Merge to VT
    .merge(
        (pd.read_csv(FP_VT_LABELS)),
        how="left",
        left_on="private_domain",
        right_on="filename",
        validate="m:1",
    ).assign(
        malicious_bool=lambda df_: np.where(df_["malicious"] >= 2, True, False),
    )
    # ==================================================
    # time
    .assign(
        visit_time_local=lambda df_: pd.to_datetime(
            df_["visit_time_local"], format="ISO8601"
        ),
        hour_of_day=lambda df_: pd.to_datetime(df_["visit_time_local"]).dt.hour
        + pd.to_datetime(df_["visit_time_local"]).dt.minute / 60
        + pd.to_datetime(df_["visit_time_local"]).dt.second / 3600,
    )
)
display(df_visits.head())
df_visits.info()

,caseid,private_domain,category,visit_time_local,visit_duration,filename,harmless,malicious,suspicious,undetected,timeout,forcepoint,sophos,bitdefender,comodo,alphamnt,malicious_bool,hour_of_day
0,205323077,google.com,Search Engines and Portals,2022-05-31 23:52:37,2,google.com,70.0,0.0,0.0,17.0,0.0,search engines and portals,search engines,searchengines,NaN,NaN,False,23.876944
1,205323077,coupons.com,"Business, Shopping",2022-06-01 01:07:35,457,coupons.com,68.0,0.0,0.0,19.0,0.0,shopping,online shopping,ads,NaN,"Marketing/Merchandising, Shopping",False,1.126389
2,205323077,google.com,Business,2022-06-01 01:15:12,55,google.com,70.0,0.0,0.0,17.0,0.0,search engines and portals,search engines,searchengines,NaN,NaN,False,1.253333
3,205323077,coupons.com,"Business, Shopping",2022-06-01 01:16:07,2225,coupons.com,68.0,0.0,0.0,19.0,0.0,shopping,online shopping,ads,NaN,"Marketing/Merchandising, Shopping",False,1.268611
4,205323077,google.com,Search Engines and Portals,2022-06-01 04:38:10,10,google.com,70.0,0.0,0.0,17.0,0.0,search engines and portals,search engines,searchengines,NaN,NaN,False,4.636111


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6297382 entries, 0 to 6297381
Data columns (total 18 columns):
 #   Column            Dtype         
---  ------            -----         
 0   caseid            int64         
 1   private_domain    object        
 2   category          object        
 3   visit_time_local  datetime64[ns]
 4   visit_duration    int64         
 5   filename          object        
 6   harmless          float64       
 7   malicious         float64       
 8   suspicious        float64       
 9   undetected        float64       
 10  timeout           float64       
 11  forcepoint        object        
 12  sophos            object        
 13  bitdefender       object        
 14  comodo            object        
 15  alphamnt          object        
 16  malicious_bool    bool          
 17  hour_of_day       float64       
dtypes: bool(1), datetime64[ns](1), float64(6), int64(2), object(8)
memory usage: 822.8+ MB


In [18]:
df = (
    # =============================================================
    pd.read_csv(FP_IND_DOMAIN)
    .assign(
        duration_min=lambda df_: df_["duration"] / 60,
        duration_hr=lambda df_: df_["duration"] / 3600,
    )
    # =============================================================
    # Merge to VT
    .merge(
        (pd.read_csv(FP_VT_LABELS)),
        how="left",
        left_on="private_domain",
        right_on="filename",
        validate="m:1",
    )
    .assign(
        malicious_bool=lambda df_: np.where(df_["malicious"] >= 2, True, False),
        malicious_visits=lambda df_: df_["malicious_bool"] * df_["visits"],
        malicious_min=lambda df_: df_["malicious_bool"] * df_["duration_min"],
        malicious_hr=lambda df_: df_["malicious_bool"] * df_["duration_hr"],
        suspicious_bool=lambda df_: df_["suspicious"].astype("bool"),
    )
    # =============================================================
    # Aggregate to ind.
    .groupby("caseid")
    .sum()
    .reset_index()
    # =============================================================
    # Get singleton proportions
    # malicious
    .merge(
        (
            df_visits.query("malicious_bool")
            .groupby(["caseid", "private_domain"])
            .size()
            .reset_index()
            .rename_column(0, "visits")
            .assign(singleton_mal=lambda df_: np.where(df_["visits"] == 1, 1, 0))
            .groupby("caseid")["singleton_mal"]
            .mean()
            .reset_index()
        ),
        how="left",
        on="caseid",
        validate="1:1",
    )
    # non-malicious
    .merge(
        (
            df_visits.query("malicious==0")
            .groupby(["caseid", "private_domain"])
            .size()
            .reset_index()
            .rename_column(0, "visits")
            .assign(singleton_nonmal=lambda df_: np.where(df_["visits"] == 1, 1, 0))
            .groupby("caseid")["singleton_nonmal"]
            .mean()
            .reset_index()
        ),
        how="left",
        on="caseid",
        validate="1:1",
    )
    # All
    .merge(
        (
            df_visits
            .groupby(["caseid", "private_domain"])
            .size()
            .reset_index()
            .rename_column(0, "visits")
            .assign(singleton=lambda df_: np.where(df_["visits"] == 1, 1, 0))
            .groupby("caseid")["singleton"]
            .mean()
            .reset_index()
        ),
        how="left",
        on="caseid",
        validate="1:1",
    )
    # =============================================================
    # Get concentration measures
    # malicious
    .merge(
        (
            df_visits.query("malicious_bool")
            .reset_index(drop=True)
            .groupby(["caseid", "private_domain"])
            .size()
            .reset_index()
            .rename_column(0, "visits")
            .groupby("caseid")["visits"]
            .apply(lambda x: Gini(x.values).g)
            .reset_index(name="gini_mal")
        ),
        how="left",
        on="caseid",
        validate="1:1",
    )
    # non-malicious
    .merge(
        (
            df_visits.query("malicious==0")
            .query("suspicious==0")
            .reset_index(drop=True)
            .groupby(["caseid", "private_domain"])
            .size()
            .reset_index()
            .rename_column(0, "visits")
            .groupby("caseid")["visits"]
            .apply(lambda x: Gini(x.values).g)
            .reset_index(name="gini_nonmal")
        ),
        how="left",
        on="caseid",
        validate="1:1",
    )
    # All
    .merge(
        (
            df_visits
            .reset_index(drop=True)
            .groupby(["caseid", "private_domain"])
            .size()
            .reset_index()
            .rename_column(0, "visits")
            .groupby("caseid")["visits"]
            .apply(lambda x: Gini(x.values).g)
            .reset_index(name="gini")
        ),
        how="left",
        on="caseid",
        validate="1:1",
    )    
    # =============================================================
    # get unique sites visited
    # malicious
    .merge(
        (
            df_visits.query("malicious_bool")
            .groupby("caseid")["private_domain"]
            .size()
            .reset_index(name="n_uniques_mal")
        ),
        how="left",
        on="caseid",
        validate="1:1",
    )
    .assign(n_uniques_mal=lambda df_: df_["n_uniques_mal"].fillna(0).astype(int))
    # non-malicious
    .merge(
        (
            df_visits.query("malicious==0")
            .query("suspicious==0")
            .groupby("caseid")["private_domain"]
            .size()
            .reset_index(name="n_uniques_nonmal")
        ),
        how="left",
        on="caseid",
        validate="1:1",
    )
    # All
    .merge(
        (
            df_visits
            .groupby("caseid")["private_domain"]
            .size()
            .reset_index(name="n_uniques")
        ),
        how="left",
        on="caseid",
        validate="1:1",
    )    
    # =============================================================
    # Average duration of visits
    # malicious
    .merge(
        (
            df_visits.query("malicious_bool")
            .groupby("caseid")["visit_duration"]
            .mean()
            .reset_index()
            .rename_column("visit_duration", "duration_mean_mal")
        ),
        how="left",
        on="caseid",
        validate="1:1",
    )    
    # non-malicious
    .merge(
        (
            df_visits.query("malicious==0")
            .query("suspicious==0")
            .groupby("caseid")["visit_duration"]
            .mean()
            .reset_index()
            .rename_column("visit_duration", "duration_mean_nonmal")
        ),
        how="left",
        on="caseid",
        validate="1:1",
    )  
    # All
    .merge(
        (
            df_visits
            .groupby("caseid")["visit_duration"]
            .mean()
            .reset_index()
            .rename_column("visit_duration", "duration_mean")
        ),
        how="left",
        on="caseid",
        validate="1:1",
    )     
    # =============================================================
    # Average time-of-day of visits
    # malicious
    .merge(
        (
            df_visits.query("malicious_bool")
            .groupby("caseid")["hour_of_day"]
            .apply(lambda x: circmean(x, high=24, low=0))
            .reset_index()
            .rename_column("hour_of_day", "tod_mean_mal")
        ),
        how="left",
        on="caseid",
        validate="1:1",
    )
    # non-malicious
    .merge(
        (
            df_visits.query("malicious==0")
            .query("suspicious==0")
            .groupby("caseid")["hour_of_day"]
            .apply(lambda x: circmean(x, high=24, low=0))
            .reset_index()
            .rename_column("hour_of_day", "tod_mean_nonmal")
        ),
        how="left",
        on="caseid",
        validate="1:1",
    )
    # All
    .merge(
        (
            df_visits
            .groupby("caseid")["hour_of_day"]
            .apply(lambda x: circmean(x, high=24, low=0))
            .reset_index()
            .rename_column("hour_of_day", "tod_mean")
        ),
        how="left",
        on="caseid",
        validate="1:1",
    )    
    # ==================================================
    # Merge to ind demo (n = 1200)
    .merge(
        (
            pd.read_csv(FP_IND_DEMO).assign(
                # https://github.com/themains/bad_domains/blob/main/data/codebook.pdf
                gender_lab=lambda df_: df_["gender"].replace({1: "Male", 2: "Female"}),
                race_lab=lambda df_: df_["race"].replace(
                    {
                        1: "White",
                        2: "Black",
                        3: "Hispanic",
                        4: "Asian",
                        5: "Other",
                        6: "Other",
                        7: "Other",
                        8: "Other",
                    }
                ),
                educ_lab=lambda df_: df_["educ"].replace(
                    {
                        1: "HS or Below",
                        2: "HS or Below",
                        3: "Some college",
                        4: "Some college",
                        5: "College",
                        6: "Postgrad",
                    }
                ),
                agegroup_lab=lambda df_: pd.cut(
                    df_["birthyr"],
                    # early baby boomers
                    # late baby boomers/ early genX
                    # genX, early millenials
                    # millenials
                    # genZ
                    bins=[1929, 1958, 1973, 1988, 1998, 2003],
                    labels=["65+", "50-64", "35-49", "25-34", "<25"],
                ),
            )
        ),
        how="left",
        on="caseid",
        validate="m:1",
    )
)
df.to_csv("../data/ind_data.csv", index=False)
display(df.head())
df.info(verbose=True)

,caseid,private_domain,duration,visits,duration_min,duration_hr,filename,harmless,malicious,suspicious,...,educ,pid3,pid7,presvote20post,inputstate,region,gender_lab,race_lab,educ_lab,agegroup_lab
0,47541,10best.com1800petmeds.com2uf4ta.net3m.com8x8.c...,263115,17194,4385.250000,73.087500,10best.com1800petmeds.com2uf4ta.net3m.com8x8.c...,37198.0,19.0,5.0,...,2,2,6,2,12,3,Female,White,HS or Below,65+
1,56565,11thstreetexpress.com1a-lab.neta-mo.netactivem...,187793,11479,3129.883333,52.164722,11thstreetexpress.com1a-lab.neta-mo.netactivem...,22541.0,10.0,3.0,...,3,3,5,2,17,2,Female,White,Some college,65+
2,203271,acop.comaepohio.comalchemer.comalldayidreamabo...,94510,6540,1575.166667,26.252778,acop.comaepohio.comalchemer.comalldayidreamabo...,10298.0,8.0,2.0,...,6,1,2,-1,54,3,Female,White,Postgrad,35-49
3,216457,aol.comaolmail.comaxonix.combackup.combing.com...,52109,1770,868.483333,14.474722,aol.comaolmail.comaxonix.combackup.combing.com...,3295.0,1.0,0.0,...,2,2,7,2,27,2,Female,White,HS or Below,35-49
4,257495,123freesolitaire.com1800petmeds.com4dex.io7sea...,188945,10012,3149.083333,52.484722,123freesolitaire.com1800petmeds.com4dex.io7sea...,19151.0,33.0,6.0,...,1,1,1,1,15,4,Female,Other,HS or Below,65+


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1134 entries, 0 to 1133
Data columns (total 50 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   caseid                1134 non-null   int64   
 1   private_domain        1134 non-null   object  
 2   duration              1134 non-null   int64   
 3   visits                1134 non-null   int64   
 4   duration_min          1134 non-null   float64 
 5   duration_hr           1134 non-null   float64 
 6   filename              1134 non-null   object  
 7   harmless              1134 non-null   float64 
 8   malicious             1134 non-null   float64 
 9   suspicious            1134 non-null   float64 
 10  undetected            1134 non-null   float64 
 11  timeout               1134 non-null   float64 
 12  forcepoint            1134 non-null   object  
 13  sophos                1134 non-null   object  
 14  bitdefender           1134 non-null   object  
 15  como